# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 5.65MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
#!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [4]:
%%capture
!pip install langchain_huggingface
!pip install langchain
!pip install langchain-community
!pip install langchain_chroma
!pip install Chroma
!pip install langchain_groq
!pip install LLMChain
!pip install -qU langchain-huggingface

In [61]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [7]:
df =pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [18]:
df.index

RangeIndex(start=0, stop=104, step=1)

In [44]:
markdown_text = []

for i in df.index:
    violation = "المخالفة:"+ df['المخالفة'][i]+" - "+ df['الغرامة'][i]
    markdown_text.append(violation)

for item in violations:
    print(item)

المخالفة:قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:عدم وجود تأمين ساري للمركبة. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:وقوف المركبة في أماكن غير مخصصة للوقوف. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. - الغرامة:الغرامة المالية 100 - 150 ريال
المخالفة:عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار. - الغرامة:الغرامة المالية 150 - 300 ريال
المخالفة:الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً. - الغرامة:الغرامة المالية 150 - 300 ريال
المخالفة:قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركب

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [45]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)

# markdown_text="\n".join(markdown_text) # Join all markDown to be in one string
# chunks = text_splitter.split_text(markdown_text)

In [46]:
# for i in chunks:
#   print(i)

المخالفة:قيادة المركبة في الأسواق التي لا يسمح
لا يسمح بالقيادة فيها. - الغرامة المالية 100 -
100 - 150 ريال
المخالفة:ترك المركبة مفتوحة وفي وضع التشغيل بعد
بعد مغادرتها. - الغرامة المالية 100 - 150 ريال
المخالفة:عدم وجود تأمين ساري للمركبة. - الغرامة
- الغرامة المالية 100 - 150 ريال
المخالفة:عبور المشاة للطرق من غير الأماكن المخصصة
المخصصة لهم. - الغرامة المالية 100 - 150 ريال
المخالفة:عدم تقيد المشاة بالإشارات الخاصة بهم. -
بهم. - الغرامة المالية 100 - 150 ريال
المخالفة:وقوف المركبة في أماكن غير مخصصة للوقوف.
للوقوف. - الغرامة المالية 100 - 150 ريال
المخالفة:عدم إعطاء أفضلية المرور للمشاة أثناء
أثناء عبورهم في المسارات المخصصة لهم. - الغرامة
- الغرامة المالية 100 - 150 ريال
المخالفة:عدم استخدام إشارة الالتفاف عند التحول
التحول لليمين أو اليسار أو التجاوز أو تغيير
أو تغيير المسار. - الغرامة المالية 150 - 300 ريال
المخالفة:الرجوع إلى الخلف في الطريق العام لمسافة
لمسافة تزيد على عشرين متراً. - الغرامة المالية
المالية 150 - 300 ريال
المخالفة:قيام سائق الدراجة الآلية أو العادية - أو
- أو 

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [54]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [93]:
database = Chroma.from_texts(texts= markdown_text.split('\n'), embedding = embeddings)
retriever = database.as_retriever()

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [72]:
template = """
أنت مساعد شخصي في إعطاء معلومات حول المخالفات المرورية. سيتم إعطاءك مجموعة قوانين ويجب عليك إجابة الأسئلة من خلالها.

المخالفة: {violation}
السياق:
{context}

 ما هي غرامة المخالفة؟ وكيف يمكن للسائق تجنب هذه المخالفة؟
"""

rag_prompt_template = PromptTemplate(
    input_variables=["violation", "context"],
    template=template
)

## Step 7: Initialize the Language Model

In [73]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [74]:
MODEL = LLMChain(llm=llm,
                 prompt=rag_prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [94]:
def query_rag(user_query):
    # vector_store = Chroma(persist_directory="/content/chroma_db")
    vector_store = retriever
    docs = vector_store.similarity_search(query=user_query, k=2)

    context = "\n".join([doc.page_content for doc in docs])

    result = MODEL.invoke({"violation": user_query, "context": context})

    return result

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [95]:
retriever = database.as_retriever(search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("ماهي الغرامة على القيادة بدون رخصة؟")
retrieved_docs

[Document(metadata={}, page_content='المخالفة:الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً. - الغرامة المالية 150 - 300 ريال'),
 Document(metadata={}, page_content='المخالفة:إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح. - الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='المخالفة:المراوغة بسرعة بين المركبات على الطرق العامة. - الغرامة المالية 3000 - 6000 ريال')]